# Getting Lyrics for Each Track from Genius
The Capstone scope has changed directions some, I still want to do a skip prediction model, but instead of looking at Spotify user behavior, it will be based on whether I would skip the song or not. The classification will be if the track begins with the artist announcing their name within the first verse. Using the lyrics I will do some NLP modeling. Lyrics are pulled from Genius using the Lyricsgenius library.

### Imports

In [1]:
import pandas as pd
import numpy as np

import time   
import lyricsgenius

%load_ext email_notify_magic
# sends me an email when a cell has finished running (for long running cells)
# https://pypi.org/project/email-notify-magic/

Setting up email notifications for cell run completion, and testing it out the first time. Because of the possibility of insecurity and the need to provide the password. I decided to use a throwaway email, that was created when Pokemon Go came out.

In [2]:
%%email funnyshitthisis@gmail.com -k
# when this cell finishes running send an email. 
# (Testing it out before the cell I really need it for)

tracks_df = pd.read_csv('./data/tracks_cleaned.csv')

Type your password and press enter: ········
Type your password and press enter: ········


In [3]:
tracks_df.head()

,artists,title,track_id,popularity,explicit,release_year
0,Shawn Mendes,Wonder,5KCbr5ndeby4y4ggthdiAb,0,False,2020
1,Justin Bieber,Holy (feat. Chance The Rapper),5u1n1kITHCxxp8twBcZxWy,92,False,2020
2,24kGoldn,Mood (feat. Iann Dior),3tjFYV6RSFtuktYl3ZtYcq,100,True,2020
3,Internet Money,Lemonade,02kDW379Yfd5PzW5A6vuGt,93,True,2020
4,BLACKPINK,Bet You Wanna (feat. Cardi B),1hPkiovjTqiJAJen4uyNRg,0,False,2020


In [4]:
print(f' There are now {tracks_df.shape[0]} rows remaining')

 There are now 76338 rows remaining


In [5]:
# Empty list to hold all the lyrics for the tracks
lyrics = []

### Get Lyrics function
The function takes no arguments, and runs starting with a while loop.
- While the length of the lyrics list (above) is not equal to the length of the dataframe
- call to the lyricsgenius library
- For each track (row) in the dataframe's values
- search for song matching the track/row's 2nd column value ('title') and the 1st column value ('artist')
- try to append the result to the lyrics list
- except if there is an issue (more likely no results) then append NaN (easy to find missing values later)
- sleep 15seconds and go again.

In [6]:
# lyricsgrenius is a python library that makes API pulls from Genius alot easier, instead of scrapping the entire HTML page to
# get the lyrics this makes it seemless where all I need to provide is the name of the artist and the song title.
# https://pypi.org/project/lyricsgenius/

def get_lyrics(): #no arguments needed
    while len(lyrics) != len(tracks_df): # while the length of list is not equal to length of dataframe
        genius = lyricsgenius.Genius("2AaY5B5Iylvf3X2oQl_4Jmk74pSZ-J8VqcboMk1PO_y7N8wlN83D9a85uhNh49Id") # call to lyricsgenius
        for track in tracks_df.values: # for each track in the dataframe. I have indexed it to 5 rows for submission only
            song = genius.search_song(track[1], track[0]) # find song on Genius with matching name and artist
            try:    
                lyrics.append(song.lyrics) # append value to list
            except:
                lyrics.append(np.NAN) # if nothing to append, append NaN
        time.sleep(15) # take a catnap        

In [ ]:
%%email funnyshitthisis@gmail.com --body 'Done'
# This is where the email notification comes handy. Getting 80k + Lyrics will take a long time.

In [ ]:
%%email funnyshitthisis@gmail.com --body 'Done'
get_lyrics() # calling the function

Type your password and press enter: ········
Searching for "Wonder" by Shawn Mendes...
Done.
Searching for "Holy (feat. Chance The Rapper)" by Justin Bieber...
Done.
Searching for "Mood (feat. Iann Dior)" by 24kGoldn...
Done.
Searching for "Lemonade" by Internet Money...
Done.
Searching for "Bet You Wanna (feat. Cardi B)" by BLACKPINK...
Done.
Searching for "Head & Heart (feat. MNEK)" by Joel Corry...
Done.
Searching for "you broke me first" by Tate McRae...
Done.
Searching for "Dynamite" by BTS...
Done.
Searching for "For The Night (feat. Lil Baby & DaBaby)" by Pop Smoke...
Done.
Searching for "Laugh Now Cry Later (feat. Lil Durk)" by Drake...
Done.
Searching for "WAP (feat. Megan Thee Stallion)" by Cardi B...
Done.
Searching for "Take You Dancing" by Jason Derulo...
Done.
Searching for "Breaking Me" by Topic...
Done.
Searching for "Savage Love (Laxed – Siren Beat) [BTS Remix]" by Jawsh 685...
Done.
Searching for "Watermelon Sugar" by Harry Styles...
Done.
Searching for "my ex's best 

In [ ]:
len(lyrics)

In [ ]:
df = tracks_df[0:18477]

In [ ]:
df

In [ ]:
df['lyrics'] = lyrics

In [ ]:
df.drop_duplicates(['artists','title'],keep= 'first', inplace=True)

In [ ]:
# df.to_csv('./data/lyrics_cp.csv')

In [ ]:
len(tracks_df)

### Some Additional Cleanup
Now that I have all my lyrics I need to drop any duplicates that might have remained: 
<br> Clean up the strings as there are some obscure characters in them \n & \ </br>
<br>I will also remove the [verse 1] [chorus] parts as the artist's names are listed in there and will through my model off.</br> <br>And finally I will create a binarized column on whether the artist self-announces at the beginning of the track, I will need to extract the &feat artist name from title to add to the artist column. </br>

In [ ]:
lyrics[0] = lyrics[0].replace('\n', ' ') 

In [ ]:
lyrics[0].replace("\'", "")

In [ ]:
lyrics[10664]

In [ ]:
del lyrics[6607]

In [ ]:
len(tracks_df)

In [ ]:
tracks_df.iloc[6607]

# Drop track_id